In [73]:
import h5py
from pathlib import Path
import os
import tensorflow as tf


In [74]:
root = Path.cwd()
experiment = 'nano-4'
weight_file = None
look_for = 'best_acc'

for file in (root / experiment).iterdir():
    if look_for not in file.name:
        continue
    weight_file = file
print(weight_file)

c:\Users\otkul\ETH\vår\project\experiments\q-nano-pelican\nano-4\best_acc.weights.h5


In [75]:
# load model
from keras import models
from qkeras.utils import load_qmodel
from model import CreateModel
import yaml
import numpy as np
def load_yaml(filename):
    with open(filename, "r") as stream:
        config = yaml.load(stream, Loader=yaml.Loader)
    return config

conf = load_yaml(root / experiment / 'config.yml')
print(conf)

const = tf.shape(np.zeros((10, 4)))

my_model = CreateModel(const, conf['model'])


{'dataset': {'feature_key': 'Pmu', 'folder': '../../data/nano_data', 'label_key': 'is_signal', 'num_particles': 32}, 'hyperparams': {'batch_size': 512, 'epochs': 1000, 'lr_init': 0.01, 'patience': 15, 'val_size': -1, 'verbose': 1}, 'model': {'activation': 'relu', 'inner_product': {'data_format': 'epxpypz', 'spurions': False}, 'n_bits': 8, 'n_hidden': 2, 'n_outputs': 1, 'num_avg': 25, 'out_activation': 'linear'}, 'save_dir': 'nano', 'seed': 545944663}


In [76]:
my_model.load_weights(str(weight_file))

c:\Users\otkul\anaconda3\envs\keras2\Lib\site-packages\keras\src\constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


In [77]:


bnorms = []
denses = []
for layer in my_model.layers:
    print(layer.name)
    if 'batch_norm' in layer.name:
        bnorms.append(layer)
    if 'dense' in layer.name:
        denses.append(layer)
print(bnorms)
print(denses)

input_15
inner_product_6
q_batch_normalization_10
lineq2v2_5
q_dense_10
multiply_5
q_activation_5
q_batch_normalization_11
lineq2v0_5
q_dense_11
activation_5
[<qkeras.qnormalization.QBatchNormalization object at 0x0000021A5316C290>, <qkeras.qnormalization.QBatchNormalization object at 0x0000021A52B84ED0>]
[<qkeras.qlayers.QDense object at 0x0000021A53083B10>, <qkeras.qlayers.QDense object at 0x0000021A5340C550>]


In [85]:
bnorm1 = bnorms[1]
weights = [w for w in bnorm1.weights]
names = [w.name for w in weights]

elems = {}
for idx, name in enumerate(names):
    if 'mean' in name:
        elems['mean'] = weights[idx].numpy()
    if 'var' in name:
        elems['var'] = weights[idx].numpy()
    if 'gamma' in name:
        elems['weight'] = weights[idx].numpy()
    if 'beta' in name:
        elems['bias'] = weights[idx].numpy()

batch = np.array((elems['mean'], elems['weight']/np.sqrt(elems['var']), elems['bias']))
print(batch)

for k, v in elems.items():
    print(k, v)

[[ 0.5610447  0.5056122]
 [ 2.104964   2.2404537]
 [ 0.6903273 -0.284152 ]]
weight [1. 1.]
bias [ 0.6903273 -0.284152 ]
mean [0.5610447 0.5056122]
var [0.22568916 0.19921777]
